In [5]:
import numpy as np

from micronet import MLP, Layer
from micronet.activations import relu, softmax
from micronet.losses import cross_entropy

In [6]:
from sklearn.datasets import make_moons, make_blobs
x, y = make_moons(n_samples=100, noise=0.1)
y = np.array([[1,0] if v == 0 else [0,1] for v in y])

In [7]:
model = MLP([
    Layer(2, 16, relu),
    Layer(16, 16, relu),
    Layer(16, 2, softmax),
])

In [8]:
model.train(x, y, loss=cross_entropy, epochs=100, batch_size=30, learning_rate=0.01)

Epoch 0: Avg Loss 0.04493362630942085
Epoch 1: Avg Loss 0.039444233664737635
Epoch 2: Avg Loss 0.03844105568215489
Epoch 3: Avg Loss 0.03763868713051245
Epoch 4: Avg Loss 0.0369129510410726
Epoch 5: Avg Loss 0.03622303768976235
Epoch 6: Avg Loss 0.03565387368297143
Epoch 7: Avg Loss 0.0351053746554969
Epoch 8: Avg Loss 0.03470347040186908
Epoch 9: Avg Loss 0.03430642825916766
Epoch 10: Avg Loss 0.03392954833427239
Epoch 11: Avg Loss 0.033682681099041094
Epoch 12: Avg Loss 0.03340588664563794
Epoch 13: Avg Loss 0.03312388820976554
Epoch 14: Avg Loss 0.032938311857405776
Epoch 15: Avg Loss 0.032699100382894516
Epoch 16: Avg Loss 0.032563495668390896
Epoch 17: Avg Loss 0.032361303459539366
Epoch 18: Avg Loss 0.03223087854527251
Epoch 19: Avg Loss 0.032031272025861174
Epoch 20: Avg Loss 0.03191292608185736
Epoch 21: Avg Loss 0.03172730151902035
Epoch 22: Avg Loss 0.031628471279033774
Epoch 23: Avg Loss 0.0314477510971336
Epoch 24: Avg Loss 0.03134789656542322
Epoch 25: Avg Loss 0.031189347

In [9]:
for prediction, actual in zip(map(model, x), y):
    print(f"Prediction: {prediction.round(3)}, Actual: {actual}")

Prediction: [0.192 0.808], Actual: [0 1]
Prediction: [0.924 0.076], Actual: [1 0]
Prediction: [0. 1.], Actual: [0 1]
Prediction: [1. 0.], Actual: [1 0]
Prediction: [0.999 0.001], Actual: [1 0]
Prediction: [0.999 0.001], Actual: [1 0]
Prediction: [0. 1.], Actual: [0 1]
Prediction: [1. 0.], Actual: [1 0]
Prediction: [0.554 0.446], Actual: [0 1]
Prediction: [0.646 0.354], Actual: [1 0]
Prediction: [0. 1.], Actual: [0 1]
Prediction: [0.054 0.946], Actual: [0 1]
Prediction: [0.066 0.934], Actual: [0 1]
Prediction: [1. 0.], Actual: [1 0]
Prediction: [0.797 0.203], Actual: [1 0]
Prediction: [0. 1.], Actual: [0 1]
Prediction: [0.192 0.808], Actual: [0 1]
Prediction: [0.999 0.001], Actual: [1 0]
Prediction: [0. 1.], Actual: [0 1]
Prediction: [0.04 0.96], Actual: [0 1]
Prediction: [0.998 0.002], Actual: [1 0]
Prediction: [0.864 0.136], Actual: [1 0]
Prediction: [0.898 0.102], Actual: [1 0]
Prediction: [1. 0.], Actual: [1 0]
Prediction: [0.239 0.761], Actual: [0 1]
Prediction: [0.856 0.144], Actu